In [10]:
# ───────────────────────────────────────────────────────────────────────────────
# 📦 Standard Library Imports
# ───────────────────────────────────────────────────────────────────────────────
import os                   # For file and directory manipulation
import sys                  # To modify Python path for custom module imports
import csv                  # To handle CSV file reading/writing
import random               # For generating random numbers
import numpy as np          # Numerical operations and array handling
import pandas as pd         # DataFrame handling for structured data
import matplotlib.pyplot as plt  # Plotting and visualization

# ───────────────────────────────────────────────────────────────────────────────
# 🌍 Third-Party Library Imports
# ───────────────────────────────────────────────────────────────────────────────
import torch                # PyTorch: deep learning framework
import geopandas as gpd     # For handling geospatial data with GeoDataFrames
from torchvision.utils import save_image

# ───────────────────────────────────────────────────────────────────────────────
# 🧩 Custom Project Modules
# ───────────────────────────────────────────────────────────────────────────────

# Add custom project folder to system path to enable local module imports
sys.path.append('C:/Users/nnobi/Desktop/FIUBA/Tesis/Project')

# Import common training routines 
from project_package.utils import train_common_routines as tcr

# Import Sentinel-2 to Venus preprocessing utilities
from project_package.data_processing import sen2venus_routines as s2v

# Import general utility functions 
from project_package.utils import utils as utils

In [18]:
# Obtener ruta del script y subir un nivel a 'project'
# script_dir = os.path.dirname(os.path.abspath(__file__))  # .../project/test
script_dir = os.getcwd()
project_dir = os.path.abspath(os.path.join(script_dir, '..'))  # .../project

# Supongamos que este es tu tensor de imágenes: [N, 3, 256, 256]
N = 10
tensor_images = torch.randn(N, 3, 256, 256).short()
labels = [i % 3 for i in range(N)]  # Ejemplo de etiquetas

# Crear carpetas
img_folder = "pt_tensors"
label_folder = "pt_labels"
os.makedirs(img_folder, exist_ok=True)
os.makedirs(label_folder, exist_ok=True)

# Guardar cada imagen como .pt
for i, img in enumerate(tensor_images):
    torch.save(img, os.path.join(img_folder, f"{i:08d}.pt"))

# Guardar cada etiqueta como archivo .cls
for i, label in enumerate(labels):
    with open(os.path.join(label_folder, f"{i:08d}.cls"), "w") as f:
        f.write(str(label))

In [19]:
import tarfile

def make_pt_tar(pt_dir, labels_dir, output_tar):
    with tarfile.open(output_tar, "w") as tar:
        for filename in sorted(os.listdir(pt_dir)):
            base = os.path.splitext(filename)[0]
            pt_path = os.path.join(pt_dir, filename)
            label_path = os.path.join(labels_dir, base + ".cls")

            tar.add(pt_path, arcname=f"{base}.pt")
            if os.path.exists(label_path):
                tar.add(label_path, arcname=f"{base}.cls")
            else:
                print(f"Etiqueta no encontrada para {base}")

make_pt_tar(img_folder, label_folder, "dataset_pt.tar")


In [ ]:
import torch
import io
import webdataset as wds
from torch.utils.data import DataLoader

# Decodificador para tensores guardados como .pt (recibe bytes, convierte a Tensor)
def decode_pt(byte_data):
    tensor = torch.load(io.BytesIO(byte_data)).float()

# Dataset WebDataset desde archivo tar
dataset = (
    wds.WebDataset("dataset_pt.tar")
    .to_tuple("pt", "pt")
    .map_tuple(decode_pt, decode_pt)  # Usamos io.BytesIO para que torch.load funcione
    .shuffle(10)
    .batched(2)
)

# DataLoader sin batch_size porque WebDataset ya los hace
dataloader = DataLoader(dataset, batch_size=None)

# Loop de prueba
for images, labels in dataloader:
    print(images.shape)  # Ej: torch.Size([2, 3, 256, 256])
    print(labels)        # Ej: [0, 1]


torch.Size([2, 3, 256, 256])
tensor([0, 1])
torch.Size([2, 3, 256, 256])
tensor([1, 0])
torch.Size([2, 3, 256, 256])
tensor([2, 2])
torch.Size([2, 3, 256, 256])
tensor([0, 2])
torch.Size([2, 3, 256, 256])
tensor([1, 0])
